#Enabling interactive plots

from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())

In [ ]:
from IPython.display import Javascript
Javascript(
"""require.config({
 paths: { 
     plotly: 'https://cdn.plot.ly/plotly-latest.min'
 }
});"""
)

# Understanding Companies in Paraná

## By Santiago Chiesa

## May 2022

# Instructions


● **Preprocess the dataset**

○ Clean (preprocess all “odd” columns, and pre-process values)

○ Enrich (get data from grouping operations, maybe enrich in different ways 👀)

● Perform EDA (exploratory data analysis) on the dataset

○ Generate visualizations

○ Share your insights and conclusions

● Preferably use Python


**Open Questions (optional):**

● Any conclusions based on the number of companies created by date, month, year etc?

● Does the number of branches for a CNPJ provided on the dataset is equal to the actual
number produced by the dataset?

● What about the business activities (CNAE)? What are the main types? Can they be
aggregated into fewer groups?

● What are the differences between the cities / zip codes?

● Is it possible to catch any spatial relationships? Which visualizations would be best in
this case?

● If you were to make any model from the data, which one do you think makes sense?



# Importing libraries and reading data

In [ ]:
#Imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
import pymysql
import pandas_profiling
import re
from datetime import datetime
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
#import plotly
#plotly.offline.init_notebook_mode (connected = True)

In [ ]:
df = pd.read_csv('../data/raw_data.csv')

In [ ]:
df.head()

# Introduction


In this assignment, I elaborated an overview of companies in the state of Paraná,  Brasil. I was provided company registration data from Receita Federal.

I added data from the Brazilian Ministry of Economics, the Institute of Geography and Statistics and open source data I found in Github.

Data has been processed and analyzed using Python and Jupyter notebook. The work is stored in the following Github Repository: 

https://github.com/sanchiesa/parana_companies



# Introduction

First, I will give a brief introduction of the State of Paraná. Then, I will use company registration data to provide an overview of the companies’ landscape there. I'll focus on their geographic location, economic activity, number of openings and their social capital.  I'll be answering crucial questions to better understand companies in Paraná:


Who are they?

Where are they?

What do they do?

When and how did it all happen?

Insights will be presented from both historical current, and future perspective. 

Finally, I will summarize insights and comment on this analysis limitations, suggesting next steps for its improvement.

# Agenda

1. Overview (Who are they?)
2. Geographic Distribution (Where are they?)
3. Economic Activity (What do they do?)
4. Limitations
5. Next Steps
6. Conclusions

# Data cleaning

In [ ]:
# Data overview
df.info()

In [ ]:
# Checking data types

df.dtypes

In [ ]:
# Checking duplicates

df.duplicated().sum() / len(df)

In [ ]:
# Checking nulls

df.isna().sum() / len(df)

In [ ]:
#Checking column values
for column in df.columns:
    print(df[column].value_counts().head(20))
    print(df[column].value_counts().tail(10))
    print()
    print()

In [ ]:
# Correcting cnpj_basico column with the first 8 digits of the document_number

df['cnpj_basico'] = df.document_number.apply(lambda x: int(str(x)[0:8]))

In [ ]:
df['cnpj_basico'].value_counts()

In [ ]:
# Replacing nulls

df.nome_fantasia.fillna('Missing_Name', inplace=True)
df.email_provider.fillna('Missing_Email', inplace=True)




In [ ]:
# Replacing nome_fantasia with odd values

df.loc[df.nome_fantasia=='.', 'nome_fantasia'] = 'Missing_Name'
df.loc[df.nome_fantasia=='***', 'nome_fantasia'] = 'Missing_Name'
df.loc[df.nome_fantasia=='****', 'nome_fantasia'] = 'Missing_Name'
df.loc[df.nome_fantasia=='*****', 'nome_fantasia'] = 'Missing_Name'
df.loc[df.nome_fantasia=='******', 'nome_fantasia'] = 'Missing_Name'
df.loc[df.nome_fantasia=='*******', 'nome_fantasia'] = 'Missing_Name'
df.loc[df.nome_fantasia=='********', 'nome_fantasia'] = 'Missing_Name'

In [ ]:
# Modifying dates

# Extracting year
df.opening_date[0]

# Extracting day 

# Extracting month

# Replacing month by number with dictionary (?)

In [ ]:
# Extracting date year
df['opening_year'] = df.opening_date.apply(lambda x: x[-4:])


In [ ]:
# Extracting date day
df['opening_day'] = df.opening_date.apply(lambda x: x[0:2])


In [ ]:
# Extracting date month
df['opening_month']= df.opening_date.apply(lambda x: "".join(re.findall("[a-zA-Z]+", x)).strip('de'))

In [ ]:
# Creating month names list
month_names = list(df.opening_month.value_counts().index)
 

In [ ]:
# Creating month numbers list
month_numbers= ['03', '05', '02', '01', '04', '08', '06', '07', '10', '09', '11', '12']

In [ ]:
# Creating function to replace values
def replacer(to_replace, list_1, list_2):
    for i in range(len(list_1)):
        if to_replace == list_1[i]:
            to_replace = list_2[i]
    return to_replace
    

In [ ]:
# Testing function
foo = 'julho'
replacer(foo, month_names, month_numbers)

In [ ]:
# Replacing month names for numbers
df['opening_month'] = df.opening_month.apply(lambda x: replacer(x, month_names, month_numbers))

In [ ]:
df['opening_month_year'] = df.opening_month + '-' + df.opening_year

In [ ]:
# Creating clean date column
df['opening_date_clean'] = df.opening_year + '/' + df.opening_month + '/' + df.opening_day

In [ ]:
# Changing to date time format
df['opening_date_clean'] = df['opening_date_clean'].apply(lambda x: datetime.strptime(x, '%Y/%m/%d'))

In [ ]:
df

In [ ]:
# Creating total socios and total branches columns

df['total_socios'] = df.total_branches_and_socios.apply(lambda x: re.findall(r'\d+', x)[0])

df['total_branches'] = df.total_branches_and_socios.apply(lambda x: re.findall(r'\d+', x)[1])

In [ ]:
# Transforming total socios and total branches type to int

df['total_socios'] = df.total_socios.apply(lambda x: int(x))
df['total_branches'] = df.total_branches.apply(lambda x: int(x))

In [ ]:
df

In [ ]:
# Uppercasing email provider's column
df.email_provider = df.email_provider.apply(lambda x: x.upper())

In [ ]:
# Creating a new column only with the email provider's company nam

df['email_provider_company'] = df.email_provider.apply(lambda x: x.split('.')[0]) 

In [ ]:
df

In [ ]:
# Checking that all cities belong to Paraná
df['foo'] = df.city_state.apply(lambda x: x[-4:])

df.foo.value_counts()

In [ ]:
df.drop(columns='foo', inplace=True)

In [ ]:
# Creating city '-PR' since it's redundant info

df['city'] = df.city_state.apply(lambda x: x[:-5])

In [ ]:
df.city.value_counts()

In [ ]:
# Transforming capital_social to integer

df['capital_social'] = df['capital_social'].apply(lambda x: int(x))

# Data Enrichment

In [ ]:
# Adding counter row

df['counter'] = 1

In [ ]:
# Economic activity grouping

df.cnae.value_counts().tail(20)

In [ ]:
df.loc[df.cnae==729401]

In [ ]:
len(df.cnae.unique())

In [ ]:
df.cnae.nunique()

In [ ]:
df.cnae_description.nunique()

In [ ]:
# Importing CNAE detailed category data
# Source: https://www.ibge.gov.br/estatisticas/metodos-e-classificacoes/classificacoes-e-listas-estatisticas/9078-classificacao-nacional-de-atividades-economicas.html?=&t=resultados
cnae_df = pd.read_excel('../data/cnae_detail.xls')

In [ ]:
# Renaming columns
cnae_df.columns=['cnae_section', 'cnae_division', 'cnae_group', 'cnae_class', 'cnae_subclass', 'denomination']

In [ ]:
# Keeping only numerical values fore numerical code groupings:
for col in cnae_df.columns[1:-1]:    
    cnae_df[col] = cnae_df[col].apply(lambda x: "".join(re.findall(r'\d+', str(x))))

In [ ]:
cnae_df

In [ ]:
# Examining observation with 6 digit CENAE
df.loc[df.cnae==133411]

In [ ]:
# Checking value in CNAE dataframe
cnae_df.loc[cnae_df.denomination == 'Cultivo de pêssego']

In [ ]:
# Transforming CNAE to string
df.cnae = df.cnae.apply(lambda x: str(x))
# Creating a column to flag 6 digits CNAE
df['foo'] = df.cnae.apply(lambda x: len(x))

In [ ]:
# Checking results
df.foo.value_counts()

In [ ]:
# Adding a 0 at the begining of 6 digit CNAEs
df.loc[df.foo==6, 'cnae'] = '0'+ df.cnae

In [ ]:
# Checking results
df['foo'] = df.cnae.apply(lambda x: len(x))
df.foo.value_counts()

In [ ]:
# Dropping temporal column
df.drop(columns='foo', inplace=True)

In [ ]:
# Creating CNAE division, group and class columns
df['cnae_division'] = df.cnae.apply(lambda x: x[0:2])
df['cnae_group'] = df.cnae.apply(lambda x: x[0:3])
df['cnae_class'] = df.cnae.apply(lambda x: x[0:5])

In [ ]:
# Creating dictionary mapping CNAE sections to divisions
cnae_section_division = {'A': ['01', '02', '03'], 
                         'B': ['05', '06', '07', '08', '09'], 
                         'C':list(range(10, 34)), 
                         'D':[35], 
                         'E':list(range(36,40)), 
                         'F':list(range(41,44)), 
                         'G':list(range(45,48)), 
                         'H':list(range(49,54)), 
                         'I':list(range(55,57)), 
                         'J':list(range(58,64)), 
                         'K':list(range(64,67)), 
                         'L':[68], 
                         'M':list(range(69, 76)), 
                         'N':list(range(77, 83)), 
                         'O':[84], 
                         'P':[85],
                         'Q':list(range(86,89)),
                         'R': list(range(90,94)), 
                         'S':list(range(94,97)), 
                         'T':[97], 
                         'U':[99]}

In [ ]:
# Transforming dictionary values from int to strings
for i in cnae_section_division:
    for j in range(len(cnae_section_division[i])):
        cnae_section_division[i][j] = str(cnae_section_division[i][j])

In [ ]:
# Defining function for using the dictionary to map our dataframe's cnae_division to secitons
def dict_imputer(to_check, dictionary):
    for i in dictionary:
        if to_check in dictionary[i]:
            return i

In [ ]:
# Creating our CNAE section column, using the dictionary and function defined above
df['cnae_section'] = df.cnae_division.apply(lambda x: dict_imputer(x, cnae_section_division))

In [ ]:
# Checking nulls

df.isna().sum() / len(df)

In [ ]:
# Merging dataframes to obtain CNAE descriptions for every CNAE grouping
for col in cnae_df.columns[0:-2]:
    foo_df = pd.DataFrame(cnae_df[[col, 'denomination']])
    df= df.merge(foo_df, how='left', on= col)
    df.rename(columns={'denomination': col + '_description'}, inplace=True)

In [ ]:
df

In [ ]:
# Checking nulls

df.isna().sum() / len(df)

In [ ]:
# Renaming cnae_subclass_description column
df.rename(columns = {'cnae_description':'cnae_subclass_description'}, inplace=True)

In [ ]:
df.columns

In [ ]:
df.head(1)

In [ ]:
# Adding coordinates data
# Source: https://github.com/kelvins/Municipios-Brasileiros
muni_df = pd.read_csv('../data/municipios.csv')


In [ ]:
muni_df.head(1)

In [ ]:
#Keeping relevant columns
muni_df= muni_df[['codigo_ibge', 'latitude', 'longitude']]

In [ ]:
#Merging datasets to add coordinates information

df = df.merge(muni_df, how='left', left_on='city_code', right_on='codigo_ibge')

In [ ]:
# Checking nulls

df.isna().sum() / len(df)

# Data Analysis

## EDA

In [ ]:
# Grouping data by month
df_grouped_time = df.groupby([df.opening_year]).agg({'capital_social': ['count', 'sum',  'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_time.columns = ['openings',
                            'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Last 50 years overview

df_grouped_time.tail(50)

In [ ]:
# Grouping data by city
df_grouped_city = df.groupby([df.city]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_city.columns = ['openings',
                           'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 cities 
df_grouped_city.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_city.head(10)

In [ ]:
# Grouping data by zip_code with their cities
df_grouped_zip = df.groupby([df.zip_code, df.city]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_zip.columns = ['openings',
                          'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 zip codes with their cities
df_grouped_zip.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_zip.head(10)

In [ ]:
# Grouping data by city_code with their cities and coordinates
df_grouped_cc = df.groupby([df.city_code, df.city, df.latitude, df.longitude]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cc.columns = ['openings',
                          'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 zip codes with their cities
df_grouped_cc.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_cc.head(10)

In [ ]:
# Grouping data by cnae_section
df_grouped_cnae_sec = df.groupby([df.cnae_section_description]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cnae_sec.columns = ['openings',
                               'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

df_grouped_cnae_sec.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_cnae_sec.head(10)

In [ ]:
# Grouping data by cnae_division
df_grouped_cnae_div = df.groupby([df.cnae_division_description]).agg({'capital_social': ['count', 'sum','mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cnae_div.columns = ['openings',
                               'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 cenae divisions
df_grouped_cnae_div.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_cnae_div.head(10)

In [ ]:
# Grouping data by cnae_group
df_grouped_cnae_gr = df.groupby([df.cnae_group_description]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cnae_gr.columns = ['openings',
                              'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 cenae groups
df_grouped_cnae_gr.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_cnae_gr.head(10)

In [ ]:
# Grouping data by cnae_class
df_grouped_cnae_cl = df.groupby([df.cnae_class_description]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cnae_cl.columns = ['openings',
                              'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 classes
df_grouped_cnae_cl.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_cnae_cl.head(10)

In [ ]:
# Grouping data by cnae_sub_class
df_grouped_cnae_scl = df.groupby([df.cnae_subclass_description]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cnae_scl.columns = ['openings',
                              'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

# Top 10 cenae sub classes
df_grouped_cnae_scl.sort_values(['openings', 'capital_social_sum'], ascending=False, inplace=True)
df_grouped_cnae_scl.head(10)

In [ ]:
# Creating function to plot distributions and boxplots

def plot_dist_box(df):
    
    #Defining figure shape
    a = len(df.columns)  
    b = 2  
    c = 1  

    fig = plt.figure(figsize=(20,60))
    
    #Plotting distributions and boxplots for each variable in the dataframe
    for i in df.columns:
        plt.subplot(a, b, c)
        plt.title('{} (dist), subplot: {}{}{}'.format(i, a, b, c))
        plt.xlabel(i)
        sns.distplot(df[i])
        c = c + 1

        plt.subplot(a, b, c)
        plt.title('{} (box), subplot: {}{}{}'.format(i, a, b, c))
        plt.xlabel(i)
        plt.boxplot(x = df[i])
        c = c + 1

    return fig

In [ ]:
# Creating function to compute descriptive statistics

def descriptive_stats(df):
    
    # Computing main descriptive stats
    df_description = pd.DataFrame(df.describe().round(1))

    # Computing median
    median = pd.DataFrame(df.median().round(1)).T.rename(index={0:'median'})

    # Computing skew
    skew = pd.DataFrame(df.skew().round(1)).T.rename(index={0:'skew'})

    # Computing one table with all stats
    df_description = df_description.append([median, skew])
    
    return df_description

In [ ]:
df.columns

In [ ]:
# Defining numerical, categorical, geographical and company ID columns
df_num_col = ['capital_social', 'total_socios', 'total_branches']
df_cat_col = ['establishment_type', 'cnae_section_description', 
              'cnae_division_description','cnae_group_description', 
              'cnae_class_description', 'cnae_subclass_description']
df_geo_col = ['city_code', 'zip_code', 'city']
df_id_col = ['document_number', 'cnpj_basico', 'razao_social', 'nome_fantasia']

In [ ]:
# Plotting distributions and boxplots of numerical columns

plot_dist_box(df[df_num_col])

In [ ]:
# Computing descriptive statistics for numerical columns

descriptive_stats(df[df_num_col])

In [ ]:
df.columns 

In [ ]:
df.establishment_type.value_counts() / len(df.establishment_type)

In [ ]:
df.juri_description.value_counts() / len(df.juri_description)

## General Overview (Who are they?)

Paraná is one of the 26 states of Brazil, its capital is the city of Curitiba. Other major cities are Londrina, Maringá, and Foz do Iguaçu.


It’s home to 5.4% of the Brazilian population (11,5 M), ranking 5th in the country.


It has 6.2% of the Brazilian GDP (R $ 452,7 B), ranking 4th behind the states of Rio de Janeiro, São Paulo and Minas Gerais.

According to the data from Receita Federal (1919-2021), Paraná has 1.282.539 registered companies. 

96% of them are registered as head offices, while the remaining 4% are registered branches.

In [ ]:
juri_description_df = pd.DataFrame(df.juri_description.value_counts())

In [ ]:
juri_description_df.reset_index(inplace=True)
juri_description_df.columns= ['juri_description', 'count']

In [ ]:
juri_description_df

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df.juri_description, 
                     values=df.counter, 
                     name="N Companies"),
              1, 1)
fig.add_trace(go.Pie(labels=df.juri_description, 
                     values=df.capital_social, 
                     name="Social Capital"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="N Companies and Social Capital per Legal Entity Type",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text="N Companies", x=0.13, y=0.5, font_size=14, showarrow=False),
                 dict(text="Social Capital", x=0.87, y=0.5, font_size=14, showarrow=False)])
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(showlegend=False)


Individual companies represent 65% of the registered companies in Paraná.


However, 97% of social capital is concentrated by limited liability companies. 

## The number of companies has been growing exponentially.

In [ ]:
df_grouped_time

In [ ]:
df_grouped_time.reset_index(inplace=True)

In [ ]:
# Plotting N of Companies historical evolution
fig = px.bar(df_grouped_time.loc[df_grouped_time.opening_year<'2021'], x='opening_year', y="openings")
fig.update_layout(
    title_text="Company Openings")
fig.show()

## Social Capital is also on the rise, but with more volatility. 

In [ ]:
# Plotting Social Capital sum historical evolution
fig = px.bar(df_grouped_time.loc[df_grouped_time.opening_year<'2021'], x='opening_year', y='capital_social_sum')
fig.update_layout(
    title_text="Social Capital Sum")
fig.show()

In [ ]:
# Plotting Social Capital mean historical evolution
fig = px.bar(df_grouped_time.loc[df_grouped_time.opening_year<'2021'], x='opening_year', y='capital_social_mean')
fig.update_layout(
    title_text="Social Capital Mean")
fig.show()

## Greographic Distribution (Where are they?)

In [ ]:
# Reseting index for city code dataframe
df_grouped_cc = df_grouped_cc.reset_index()

In [ ]:
#Plotting number of company openings on a map

fig = px.scatter_mapbox(df_grouped_cc, lat="latitude", lon="longitude", color="openings", size="openings",
                        color_continuous_scale="haline", size_max=60,
                        mapbox_style="carto-positron",hover_name='city',labels={
                        "openings": "Companies"
                    })
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=525)


In [ ]:
#Plotting companies' capital sum on a map


fig = px.scatter_mapbox(df_grouped_cc, lat="latitude", lon="longitude", color="capital_social_sum", size="capital_social_sum",
                        color_continuous_scale="haline", size_max=60,
                        mapbox_style="carto-positron", hover_name='city',labels={
                        "capital_social_sum": "Capital Social"
                    })
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=525)


## N of Companies and their Social Capital in the Map

In [ ]:
#Plotting number of companies (color) and companies' capital sum (size) on a map.

fig = px.scatter_mapbox(df_grouped_cc, lat="latitude", lon="longitude", color="openings", size="capital_social_sum",
                        color_continuous_scale="haline", size_max=60,
                        mapbox_style="carto-positron", hover_name='city',labels={
                        "capital_social_sum": "Capital Social", "openings":"N Companies"
                    })
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=525)


In [ ]:
# Grouping data by city_code with their cities, coordinates and year
df_grouped_cc_year = df.groupby([df.opening_year, df.city_code, df.city, df.latitude, df.longitude]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cc_year.columns = ['openings',
                          'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

df_grouped_cc_year.reset_index(inplace=True)

## What about the last two decades?

In [ ]:
#Plotting number of companies (color) and companies' capital sum (size) on a map from 2010 till 2021

fig = px.scatter_mapbox(df_grouped_cc_year.loc[(df_grouped_cc_year.opening_year>='2010')&(df_grouped_cc_year.opening_year<='2022')],
                        lat="latitude", lon="longitude", color="openings", size="capital_social_sum",
                        color_continuous_scale="haline", size_max=60,
                        mapbox_style="carto-positron", hover_name='city', labels={
                        "capital_social_sum": "Capital Social", "openings":"N Cohidempanies"
                    })
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(width=1000, height=525)


In [ ]:
# Plotting Social Capital historical evolution in the main cities.


In [ ]:
# Selecting top 6 cities by Social Capital sum
top_6_cities = list(df_grouped_city[:6].index)

In [ ]:
# Grouping data by time and city with only the top 6 cities
df_grouped_time_6cities = df.loc[df.city.isin(top_6_cities)==True].groupby([df.opening_year, df.city]).agg({'capital_social': ['count', 'sum',  'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_time_6cities.columns = ['openings',
                            'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

In [ ]:
# Checking cities
df_grouped_time_6cities = df_grouped_time_6cities.reset_index()
df_grouped_time_6cities.reset_index().city.value_counts()

In [ ]:
# Examine data
df_grouped_time_6cities

In [ ]:
# Computing openings per city per year
openings_year_6cities = df_grouped_time_6cities.pivot_table(values='openings', index='opening_year', 
                                                      columns='city', aggfunc='sum')

## Company Openings Evolution in the Main Cities

In [ ]:
# Plotting openings per city per year
fig = px.area(openings_year_6cities.loc[openings_year_6cities.index<'2021'], 
              facet_col="city", facet_col_wrap=2)
fig.show()

In [ ]:
# Computing capital sum per city per year
capital_sum_year_6cities = df_grouped_time_6cities.pivot_table(values='capital_social_sum', index='opening_year', columns='city', aggfunc='sum')

## Social Capital Sum Evolution in the Main Cities

In [ ]:
# Plotting capital sum per city per year
fig = px.area(capital_sum_year_6cities.loc[capital_sum_year_6cities.index<'2021'], facet_col="city", facet_col_wrap=2)
fig.show()

In [ ]:
# Computing capital mean per city per year
capital_mean_year_6cities = df_grouped_time_6cities.pivot_table(values='capital_social_sum', index='opening_year', columns='city', aggfunc='mean')

## Social Capital Mean Evolution in the Main Cities

In [ ]:
# Plotting capital mean per city per year
fig = px.area(capital_mean_year_6cities.loc[capital_mean_year_6cities.index<'2021'], facet_col="city", facet_col_wrap=2)
fig.show()

In [ ]:
# Grouping data by time and city with every city
df_grouped_time_city = df.groupby([df.opening_year, df.city]).agg({'capital_social': ['count', 'sum',  'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_time_city.columns = ['openings',
                            'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

In [ ]:
# Checking cities
df_grouped_time_city = df_grouped_time_city.reset_index()
df_grouped_time_city.reset_index().city.value_counts()

In [ ]:
# Examine data
df_grouped_time_city

In [ ]:
# Computing capital sum per city in 2000 and 2021
capital_sum_year_city = df_grouped_time_city.pivot_table(values='capital_social_sum', index='opening_year', 
                                                         columns='city', aggfunc='sum')

In [ ]:
# Grouping data by city and year

# Grouping data by city and year
df_grouped_city_year = df.groupby([df.city, df.opening_year]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_city_year.columns = ['openings',
                           'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

df_grouped_city_year.reset_index(inplace=True)


In [ ]:
df_grouped_city_year

## Company openings per City in the last two decades

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='1990')&(df_grouped_city_year.opening_year<='1999')]['city'], 
                     values=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='1990')&(df_grouped_city_year.opening_year<='1999')]['openings'], 
                     name="90's"),
              1, 1)
fig.add_trace(go.Pie(labels=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='2010')&(df_grouped_city_year.opening_year<='2019')]['city'], 
                     values=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='2010')&(df_grouped_city_year.opening_year<='2019')]['openings'], 
                     name="10's"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Company Openings per City 90's vs 10's",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text="90's", x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text="10's", x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

## Social Capital Sum per City in the last two decades

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='1990')&(df_grouped_city_year.opening_year<='1999')]['city'], 
                     values=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='1990')&(df_grouped_city_year.opening_year<='1999')]['capital_social_sum'], 
                     name="90's"),
              1, 1)
fig.add_trace(go.Pie(labels=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='2010')&(df_grouped_city_year.opening_year<='2019')]['city'], 
                     values=df_grouped_city_year.loc[(df_grouped_city_year.opening_year>='2010')&(df_grouped_city_year.opening_year<='2019')]['capital_social_sum'], 
                     name="10's"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Openings' Capital Sum per City 90's vs 10's",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text="90's", x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text="10's", x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

Curitiba captures the majority of the openings and the social capital.


However, the rest of the cities are catching up.

## Economic Activity (What do they do?)

In [ ]:
# Plotting plotly pie charts
cnae_groupings = [df_grouped_cnae_sec,
                  df_grouped_cnae_div, 
                  df_grouped_cnae_gr, 
                  df_grouped_cnae_cl, 
                  df_grouped_cnae_scl]

# N of Companies per CNAE section

fig = px.pie(cnae_groupings[0].reset_index(), values='openings', 
             hover_name= cnae_groupings[0].reset_index().columns[0], 
             title='N of Companies per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# Social Capital Sum per CNAE section

fig = px.pie(cnae_groupings[0].reset_index(), values='capital_social_sum', 
             hover_name= cnae_groupings[0].reset_index().columns[0], 
             title='Social Capital per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# N of Companies per CNAE division

fig = px.pie(cnae_groupings[1].reset_index(), values='openings', 
             hover_name= cnae_groupings[1].reset_index().columns[0],
             title='N of Companies per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# Social Capital Sum per CNAE division

fig = px.pie(cnae_groupings[1].reset_index(), values='capital_social_sum', 
             hover_name= cnae_groupings[1].reset_index().columns[0], 
             title='Social Capital per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# N of Companies per CNAE group

fig = px.pie(cnae_groupings[2].reset_index(), values='openings', 
             hover_name= cnae_groupings[2].reset_index().columns[0],
             title='N of Companies per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# Social Capital Sum per CNAE group

fig = px.pie(cnae_groupings[2].reset_index(), values='capital_social_sum', 
             hover_name= cnae_groupings[2].reset_index().columns[0], 
             title='Social Capital per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# N of Companies per CNAE class

fig = px.pie(cnae_groupings[3].reset_index(), values='openings', 
             hover_name= cnae_groupings[3].reset_index().columns[0],
             title='N of Companies per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# Social Capital Sum per CNAE class

fig = px.pie(cnae_groupings[3].reset_index(), values='capital_social_sum', 
             hover_name= cnae_groupings[3].reset_index().columns[0], 
             title='Social Capital per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# N of Companies per CNAE subclass

fig = px.pie(cnae_groupings[4].reset_index(), values='openings', 
             hover_name= cnae_groupings[4].reset_index().columns[0],
             title='N of Companies per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


In [ ]:
# Social Capital Sum per CNAE subclass

fig = px.pie(cnae_groupings[4].reset_index(), values='capital_social_sum', 
             hover_name= cnae_groupings[4].reset_index().columns[0], 
             title='Social Capital per Economic Activity')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()


Businesses are spred within a wide variety of activities.


Capital is slightly more concentrated within activities, although quite diversified.

In [ ]:
cnae_groupings[3].head(6)

In [ ]:
cnae_groupings[3].sort_values('capital_social_sum', ascending=False, inplace=True)

In [ ]:
# Selecting top 6 CNAE Classes by Social Capital sum
top_6_cnae_c = list(cnae_groupings[3][:6].index)

In [ ]:
top_6_cnae_c

In [ ]:
# Grouping data by time and city with only the top 6 CNAE classes
df_grouped_time_6cnaec = df.loc[df.cnae_class_description.isin(top_6_cnae_c)==True].groupby([df.opening_year, df.cnae_class_description]).agg({'capital_social': ['count', 'sum',  'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_time_6cnaec.columns = ['openings',
                            'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

In [ ]:
df_grouped_time_6cnaec

In [ ]:
# Checking CNAE classes
df_grouped_time_6cnaec = df_grouped_time_6cnaec.reset_index()
df_grouped_time_6cnaec.reset_index().cnae_class_description.value_counts()

In [ ]:
# Examine data
df_grouped_time_6cnaec

In [ ]:
# Computing openings per CNAE class per year
openings_year_6cnaec = df_grouped_time_6cnaec.pivot_table(values='openings', index='opening_year', 
                                                      columns='cnae_class_description', aggfunc='sum')

## Openings evolution per economic activity

In [ ]:
# Plotting openings per CNAE class per year
fig = px.area(openings_year_6cnaec.loc[openings_year_6cnaec.index<'2021'],
              facet_col="cnae_class_description", facet_col_wrap=2)
fig.update_layout(showlegend=False)
# subplot titles
for anno in fig['layout']['annotations']:
    anno['text']=''

fig.show()

In [ ]:
# Computing capital sum per CNAE class per year
capital_sum_year_6cnaec = df_grouped_time_6cnaec.pivot_table(values='capital_social_sum', index='opening_year', columns='cnae_class_description', aggfunc='sum')

## Capital Sum per Economic Activity

In [ ]:
# Plotting capital sum per CNAE class per year
fig = px.area(capital_sum_year_6cnaec.loc[capital_sum_year_6cnaec.index<'2021'], 
              facet_col="cnae_class_description", facet_col_wrap=2)
fig.update_layout(showlegend=False)
# subplot titles
for anno in fig['layout']['annotations']:
    anno['text']=''

fig.show()

In [ ]:
# Computing capital mean per CNAE class per year
capital_mean_year_6cnaec = df_grouped_time_6cnaec.pivot_table(values='capital_social_sum', index='opening_year', columns='cnae_class_description', aggfunc='mean')

## Capital Mean per Economic Activity

In [ ]:
# Plotting capital mean per CNAE class per year
fig = px.area(capital_mean_year_6cnaec,
              facet_col="cnae_class_description", facet_col_wrap=2)
fig.update_layout(showlegend=False)
# subplot titles
for anno in fig['layout']['annotations']:
    anno['text']=''

fig.show()

In [ ]:
# Grouping data by time and CNAE class with every CNAE class
df_grouped_time_cnaec = df.groupby([df.opening_year, df.cnae_class_description]).agg({'capital_social': ['count', 'sum',  'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_time_cnaec.columns = ['openings',
                            'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

In [ ]:
# Checking CNAE classes
df_grouped_time_cnaec = df_grouped_time_cnaec.reset_index()
df_grouped_time_cnaec.reset_index().cnae_class_description.value_counts()

In [ ]:
# Examine data
df_grouped_time_cnaec

In [ ]:
# Computing capital sum per CNAE Class in 2000 and 2021
capital_sum_year_cnaec = df_grouped_time_cnaec.pivot_table(values='capital_social_sum', index='opening_year', 
                                                         columns='cnae_class_description', aggfunc='sum')

In [ ]:
# Grouping data by CNAE class and year

# Grouping data by CNAE class and year
df_grouped_cnaec_year = df.groupby([df.cnae_class_description, df.opening_year]).agg({'capital_social': ['count', 'sum', 'mean', 'median', 'min', 'max'], 
                                                           'total_socios': ['mean'],
                                                           'total_branches': ['mean']})

df_grouped_cnaec_year.columns = ['openings',
                           'capital_social_sum',
                            'capital_social_mean',
                            'capital_social_median',
                            'capital_social_min', 
                            'capital_social_max',
                            'total_socios_mean',
                            'total_branches_mean']

df_grouped_cnaec_year.reset_index(inplace=True)


In [ ]:
df_grouped_cnaec_year

## Company openings per Economic Activity in the last two Decades

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='1990')&(df_grouped_cnaec_year.opening_year<='1999')]['cnae_class_description'], 
                     values=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='1990')&(df_grouped_cnaec_year.opening_year<='1999')]['openings'], 
                     name="90's"),
              1, 1)
fig.add_trace(go.Pie(labels=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='2010')&(df_grouped_cnaec_year.opening_year<='2019')]['cnae_class_description'], 
                     values=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='2010')&(df_grouped_cnaec_year.opening_year<='2019')]['openings'], 
                     name="10's"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Company Openings per City 90's vs 10's",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text="90's", x=0.20, y=0.5, font_size=20, showarrow=False),
                 dict(text="10's", x=0.80, y=0.5, font_size=20, showarrow=False)])
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(showlegend=False)
fig.show()

## Capital Sum per Economic Activity in the last two Decades

In [ ]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='1990')&(df_grouped_cnaec_year.opening_year<='1999')]['cnae_class_description'], 
                     values=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='1990')&(df_grouped_cnaec_year.opening_year<='1999')]['capital_social_sum'], 
                     name="90's"),
              1, 1)
fig.add_trace(go.Pie(labels=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='2010')&(df_grouped_cnaec_year.opening_year<='2019')]['cnae_class_description'], 
                     values=df_grouped_cnaec_year.loc[(df_grouped_cnaec_year.opening_year>='2010')&(df_grouped_cnaec_year.opening_year<='2019')]['capital_social_sum'], 
                     name="10's"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Capital Sum per City 90's vs 10's",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text="90's", x=0.20, y=0.5, font_size=20, showarrow=False),
                 dict(text="10's", x=0.80, y=0.5, font_size=20, showarrow=False)])
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(showlegend=False)
fig.show()

Both company openings and social capital distribution can vary significantly between decades. Some activities might get higher traction than others.


For example: Food Supermarkets were the star of the nineties, with 35% of the new companies' Social Capital.

    
However, it only reached the 6% of the cake in the next decade.

New companies' are being more diverse in terms of economic activity.

Likewise, social capital is less concentrated within economic activities. 

# Limitations



    

Time constraints resulted in scope limitations.


Limited research and domain knowledge. 

Data quality & reliability casts some doubts:
    
Are companies still active?
    
In which currency is money expressed? 

What about inflation?

Capital does not imply performance, revenue, or profit.


What about future predictions?

## Suggested Next Steps


Conduct extensive research guided by business goals.


Get more data and ensure its quality and reliability.


Cast code into functions to make it more efficient and re-usable to other cases.


Improve visualizations, make them self explanatory.

Data should be enriched to generate more and better insights.


Presentation should be improved to ensure effective communication.


Organize visualizations in a Dashboard. 


Model data to predict future behavior of variables of interest.

# Conclusions

- Individual companies represent 65% of the registered companies in Paraná.


- However, 97% of social capital is concentrated by limited liability companies. 


- The number of companies has been growing exponentially.


- Social Capital is also on the rise, but with more volatility. 


- Curitiba captures the majority of the openings and the social capital.


- However, the rest of the cities are catching up.

# Conclusions

- Businesses are spread within a wide variety of activities.


- Capital is slightly more concentrated within activities, although quite diversified.



- Both company openings and social capital distribution can vary significantly between decades. Some activities might get higher traction than others.


- New companies are being more diverse in terms of economic activity.


- Likewise, social capital is less concentrated within economic activities. 

## Thank you for your time!



Any questions?

## Command to make the presentation:



jupyter nbconvert notebooks/Analysis_final.ipynb --to slides --no-prompt --TagRemovePreprocessor.remove_input_tags={\"hide\"} --post serve 
